In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score, precision_recall_curve, average_precision_score
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline
from collections import Counter
import joblib
import warnings
warnings.filterwarnings('ignore')

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']  # 设置中文字体
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题
plt.style.use('seaborn-v0_8-whitegrid')  # 使用seaborn样式

# 创建中文字体对象
font_path = 'C:/Windows/Fonts/simhei.ttf'
if os.path.exists(font_path):
    chinese_font = fm.FontProperties(fname=font_path)
    print("使用SimHei字体文件路径")
else:
    # 回退到rcParams方法
    plt.rcParams['font.sans-serif'] = ['SimHei']
    plt.rcParams['axes.unicode_minus'] = False
    chinese_font = None
    print("使用rcParams设置SimHei字体")

# 创建输出目录
output_dir = 'dengue_classification_results'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 读取处理过的数据
print("=== 开始加载数据 ===")
df = pd.read_csv('../pre_data/processed_dengue_data.csv')

# 确保日期列为datetime类型
df['Date'] = pd.to_datetime(df['Date'])

print(f"原始数据形状: {df.shape}")
print(f"数据时间范围: {df['Date'].min()} 至 {df['Date'].max()}")
print("数据加载完成！\n")


使用SimHei字体文件路径
=== 开始加载数据 ===
原始数据形状: (1020, 8)
数据时间范围: 2016-01-01 00:00:00 至 2020-12-01 00:00:00
数据加载完成！



In [2]:
# 1. 准备分类模型特征和目标
print("1. 准备分类模型特征和目标")

# 确保数据按地区和日期排序
df = df.sort_values(['Region', 'Date'])

# 1.1 创建滞后特征
print("  创建滞后特征...")
for lag in range(1, 4):
    df[f'Lag{lag}_Cases'] = df.groupby('Region')['Dengue_Cases'].shift(lag)

# 1.2 创建移动平均特征
print("  创建移动平均特征...")
df['MA3_Cases'] = df.groupby('Region')['Dengue_Cases'].transform(
    lambda x: x.rolling(window=3, min_periods=1).mean())
df['MA6_Cases'] = df.groupby('Region')['Dengue_Cases'].transform(
    lambda x: x.rolling(window=6, min_periods=1).mean())
df['MA12_Cases'] = df.groupby('Region')['Dengue_Cases'].transform(
    lambda x: x.rolling(window=12, min_periods=1).mean())

# 1.3 创建季节性特征
print("  创建季节性特征...")
df['Month_Sin'] = np.sin(2 * np.pi * df['MonthNum'] / 12)
df['Month_Cos'] = np.cos(2 * np.pi * df['MonthNum'] / 12)

# 1.4 创建趋势特征
print("  创建趋势特征...")
df['Year_Num'] = df['Year'] - df['Year'].min()  # 从0开始的年份数值

# 1.5 添加增长率特征
print("  创建增长率特征...")
df['MoM_Growth'] = df.groupby('Region')['Dengue_Cases'].pct_change()
df['YoY_Growth'] = df.groupby(['Region', 'MonthNum'])['Dengue_Cases'].pct_change()

# 1.6 创建地区编码（独热编码）
print("  创建地区编码...")
region_dummies = pd.get_dummies(df['Region'], prefix='region')
df = pd.concat([df, region_dummies], axis=1)

# 1.7 处理缺失值
print("  处理缺失值...")
# 对于增长率特征，将无穷大替换为NaN然后替换为0
for col in ['MoM_Growth', 'YoY_Growth']:
    df[col] = df[col].replace([np.inf, -np.inf], np.nan)

# 填充缺失值
num_cols = df.select_dtypes(include=[np.number]).columns
df[num_cols] = df[num_cols].fillna(0)

# 1.8 创建爆发标签（基于95%分位数阈值）
print("  创建爆发标签...")
# 计算各地区的95%分位数
percentile_95 = df.groupby('Region')['Dengue_Cases'].transform(
    lambda x: x.quantile(0.95))

# 创建二元爆发标签，1表示爆发，0表示无爆发
df['Outbreak'] = (df['Dengue_Cases'] > percentile_95).astype(int)

# 检查爆发与非爆发样本数量
outbreak_counts = df['Outbreak'].value_counts()
print(f"  样本分布: 无爆发样本: {outbreak_counts[0]}, 爆发样本: {outbreak_counts[1]}")
print(f"  爆发率: {outbreak_counts[1] / len(df) * 100:.2f}%")

1. 准备分类模型特征和目标
  创建滞后特征...
  创建移动平均特征...
  创建季节性特征...
  创建趋势特征...
  创建增长率特征...
  创建地区编码...
  处理缺失值...
  创建爆发标签...
  样本分布: 无爆发样本: 969, 爆发样本: 51
  爆发率: 5.00%


In [3]:
# 2. 准备训练、验证和测试数据
print("\n2. 准备训练、验证和测试数据")

# 2.1 定义特征和目标列
target_col = 'Outbreak'

# 基础特征集
feature_cols = [
    # 时间特征
    'Year_Num', 'MonthNum', 'Month_Sin', 'Month_Cos',
    # 滞后特征
    'Lag1_Cases', 'Lag2_Cases', 'Lag3_Cases',
    # 移动平均特征
    'MA3_Cases', 'MA6_Cases', 'MA12_Cases',
    # 增长率特征
    'MoM_Growth', 'YoY_Growth'
]

# 添加地区编码特征
region_cols = [col for col in df.columns if col.startswith('region_')]
feature_cols.extend(region_cols)

print(f"  选择的特征数量: {len(feature_cols)}")
print(f"  前5个特征: {feature_cols[:5]}")

# 2.2 按时间分割数据
# 使用最后12个月作为测试集
test_start_date = df['Date'].max() - pd.DateOffset(months=11)
print(f"  测试集开始日期: {test_start_date}")

# 分割数据
train_df = df[df['Date'] < test_start_date].copy()
test_df = df[df['Date'] >= test_start_date].copy()

print(f"  训练集形状: {train_df.shape}")
print(f"  测试集形状: {test_df.shape}")

# 2.3 准备X和y
X_train = train_df[feature_cols]
y_train = train_df[target_col]

X_test = test_df[feature_cols]
y_test = test_df[target_col]



2. 准备训练、验证和测试数据
  选择的特征数量: 29
  前5个特征: ['Year_Num', 'MonthNum', 'Month_Sin', 'Month_Cos', 'Lag1_Cases']
  测试集开始日期: 2020-01-01 00:00:00
  训练集形状: (816, 37)
  测试集形状: (204, 37)


In [4]:
# 3. 处理样本不平衡问题
print("\n3. 处理样本不平衡问题")

# 3.1 检查训练集中的类别不平衡
train_class_counts = Counter(y_train)
print(f"  训练集类别分布: {train_class_counts}")
imbalance_ratio = train_class_counts[0] / train_class_counts[1]
print(f"  不平衡比例 (无爆发/爆发): {imbalance_ratio:.2f}")

# 3.2 创建混合采样策略
print("  实现混合采样策略...")
# 先进行下采样然后进行SMOTE过采样
# 下采样比例保持2:1（无爆发:爆发）
sampling_strategy_under = {0: train_class_counts[1] * 2, 1: train_class_counts[1]}
# SMOTE采样平衡类别
sampling_strategy_over = {0: train_class_counts[1] * 2, 1: train_class_counts[1] * 2}

# 创建采样流水线
sampling_pipeline = ImbPipeline(steps=[
    ('under', RandomUnderSampler(sampling_strategy=sampling_strategy_under, random_state=42)),
    ('over', SMOTE(sampling_strategy=sampling_strategy_over, random_state=42))
])

# 应用采样策略
X_resampled, y_resampled = sampling_pipeline.fit_resample(X_train, y_train)

# 检查重采样后的类别分布
resampled_class_counts = Counter(y_resampled)
print(f"  重采样后的类别分布: {resampled_class_counts}")
print(f"  重采样后的不平衡比例: {resampled_class_counts[0] / resampled_class_counts[1]:.2f}")


3. 处理样本不平衡问题
  训练集类别分布: Counter({0: 765, 1: 51})
  不平衡比例 (无爆发/爆发): 15.00
  实现混合采样策略...
  重采样后的类别分布: Counter({0: 102, 1: 102})
  重采样后的不平衡比例: 1.00


In [5]:
# 4. 实现随机森林分类器
print("\n4. 实现随机森林分类器")

# 4.1 创建基础分类器
print("  创建基础随机森林分类器...")
base_clf = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='sqrt',
    random_state=42,
    class_weight='balanced',  # 使用类别权重进一步处理不平衡
    n_jobs=-1
)

# 4.2 训练模型
print("  训练基础分类器...")
base_clf.fit(X_resampled, y_resampled)

# 4.3 预测测试集
print("  预测测试集...")
y_pred = base_clf.predict(X_test)
y_pred_proba = base_clf.predict_proba(X_test)[:, 1]  # 获取正类的概率

# 4.4 初步评估模型
print("  初步评估模型...")
# 混淆矩阵
conf_matrix = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = conf_matrix.ravel()

# 计算评估指标
accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

print(f"  混淆矩阵:\n{conf_matrix}")
print(f"  准确率: {accuracy:.4f}")
print(f"  精确率: {precision:.4f}")
print(f"  召回率: {recall:.4f}")
print(f"  F1分数: {f1:.4f}")
print(f"  特异性: {specificity:.4f}")

# 4.5 保存基础模型
joblib.dump(base_clf, f'{output_dir}/base_rf_classifier.pkl')
print(f"  基础模型已保存至: {output_dir}/base_rf_classifier.pkl")


4. 实现随机森林分类器
  创建基础随机森林分类器...
  训练基础分类器...
  预测测试集...
  初步评估模型...
  混淆矩阵:
[[203   1]
 [  0   0]]
  准确率: 0.9951
  精确率: 0.0000
  召回率: 0.0000
  F1分数: 0.0000
  特异性: 0.9951
  基础模型已保存至: dengue_classification_results/base_rf_classifier.pkl


In [6]:
# 1. 评估分类模型性能
print("1. 评估分类模型性能")

# 1.1 计算并显示详细分类报告
print("  详细分类报告:")
class_report = classification_report(y_test, y_pred)
print(class_report)

# 保存分类报告到文件
with open(f'{output_dir}/classification_report.txt', 'w') as f:
    f.write(class_report)

# 1.2 绘制ROC曲线
print("  绘制ROC曲线...")
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
roc_auc = roc_auc_score(y_test, y_pred_proba)

plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC曲线 (AUC = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])

if chinese_font is not None:
    plt.title('接收者操作特征曲线 (ROC)', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('假阳性率 (1 - 特异性)', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('真阳性率 (敏感度)', fontproperties=chinese_font, fontsize=12)
    plt.legend(loc="lower right", prop=chinese_font)
else:
    plt.title('接收者操作特征曲线 (ROC)', fontsize=16)
    plt.xlabel('假阳性率 (1 - 特异性)', fontsize=12)
    plt.ylabel('真阳性率 (敏感度)', fontsize=12)
    plt.legend(loc="lower right")

plt.grid(True)
plt.tight_layout()
plt.savefig(f'{output_dir}/roc_curve.png', dpi=300)
print(f"  保存图表：{output_dir}/roc_curve.png")
plt.close()

# 1.3 绘制精确率-召回率曲线
print("  绘制精确率-召回率曲线...")
precision_curve, recall_curve, _ = precision_recall_curve(y_test, y_pred_proba)
pr_auc = average_precision_score(y_test, y_pred_proba)

plt.figure(figsize=(10, 8))
plt.plot(recall_curve, precision_curve, color='blue', lw=2, 
         label=f'精确率-召回率曲线 (AP = {pr_auc:.3f})')
plt.axhline(y=outbreak_counts[1] / len(df), color='red', linestyle='--', 
            label=f'基准 (类别分布 = {outbreak_counts[1] / len(df):.3f})')

if chinese_font is not None:
    plt.title('精确率-召回率曲线', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('召回率', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('精确率', fontproperties=chinese_font, fontsize=12)
    plt.legend(loc="lower left", prop=chinese_font)
else:
    plt.title('精确率-召回率曲线', fontsize=16)
    plt.xlabel('召回率', fontsize=12)
    plt.ylabel('精确率', fontsize=12)
    plt.legend(loc="lower left")

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.grid(True)
plt.tight_layout()
plt.savefig(f'{output_dir}/precision_recall_curve.png', dpi=300)
print(f"  保存图表：{output_dir}/precision_recall_curve.png")
plt.close()

# 1.4 绘制混淆矩阵热图
print("  绘制混淆矩阵热图...")
plt.figure(figsize=(10, 8))
conf_matrix_normalized = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
sns.heatmap(conf_matrix_normalized, annot=conf_matrix, fmt='d', cmap='Blues', 
            xticklabels=['无爆发', '爆发'], yticklabels=['无爆发', '爆发'])

if chinese_font is not None:
    plt.title('混淆矩阵', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('预测标签', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('真实标签', fontproperties=chinese_font, fontsize=12)
else:
    plt.title('混淆矩阵', fontsize=16)
    plt.xlabel('预测标签', fontsize=12)
    plt.ylabel('真实标签', fontsize=12)

plt.tight_layout()
plt.savefig(f'{output_dir}/confusion_matrix.png', dpi=300)
print(f"  保存图表：{output_dir}/confusion_matrix.png")
plt.close()


1. 评估分类模型性能
  详细分类报告:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       204
           1       0.00      0.00      0.00         0

    accuracy                           1.00       204
   macro avg       0.50      0.50      0.50       204
weighted avg       1.00      1.00      1.00       204

  绘制ROC曲线...
  保存图表：dengue_classification_results/roc_curve.png
  绘制精确率-召回率曲线...
  保存图表：dengue_classification_results/precision_recall_curve.png
  绘制混淆矩阵热图...
  保存图表：dengue_classification_results/confusion_matrix.png


In [7]:
# 2. 分析特征重要性
print("\n2. 分析特征重要性")

# 2.1 获取特征重要性
feature_importances = base_clf.feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': feature_importances
}).sort_values('Importance', ascending=False)

# 2.2 保存特征重要性
feature_importance_df.to_csv(f'{output_dir}/feature_importance.csv', index=False)
print(f"  特征重要性已保存至: {output_dir}/feature_importance.csv")

# 2.3 可视化特征重要性
print("  可视化特征重要性...")
plt.figure(figsize=(12, 10))
top_n = min(15, len(feature_cols))  # 取前15个特征或全部特征
top_features = feature_importance_df.head(top_n)

# 绘制条形图
ax = sns.barplot(x='Importance', y='Feature', data=top_features)

if chinese_font is not None:
    plt.title('随机森林特征重要性（前15个）', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('重要性得分', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('特征', fontproperties=chinese_font, fontsize=12)
else:
    plt.title('随机森林特征重要性（前15个）', fontsize=16)
    plt.xlabel('重要性得分', fontsize=12)
    plt.ylabel('特征', fontsize=12)

plt.tight_layout()
plt.savefig(f'{output_dir}/feature_importance.png', dpi=300)
print(f"  保存图表：{output_dir}/feature_importance.png")
plt.close()


2. 分析特征重要性
  特征重要性已保存至: dengue_classification_results/feature_importance.csv
  可视化特征重要性...
  保存图表：dengue_classification_results/feature_importance.png


In [8]:
# 3. 优化模型参数
print("\n3. 优化模型参数")

# 3.1 定义参数搜索空间
print("  定义参数搜索空间...")
param_distributions = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [5, 8, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

# 3.2 设置时间序列交叉验证
tscv = TimeSeriesSplit(n_splits=3)

# 3.3 执行随机搜索
print("  执行随机参数搜索（这可能需要一些时间）...")
search = RandomizedSearchCV(
    estimator=RandomForestClassifier(random_state=42, class_weight='balanced'),
    param_distributions=param_distributions,
    n_iter=20,  # 尝试20组参数组合
    scoring='f1',  # 优化F1分数
    cv=tscv,
    random_state=42,
    n_jobs=-1,
    verbose=1
)

# 在重采样的数据上训练
search.fit(X_resampled, y_resampled)

best_params = search.best_params_
best_score = search.best_score_

print(f"  最佳F1分数: {best_score:.4f}")
print("  最佳参数:")
for param, value in best_params.items():
    print(f"    {param}: {value}")

# 3.4 使用最佳参数训练最终模型
print("  使用最佳参数训练最终模型...")
final_clf = RandomForestClassifier(
    **best_params,
    random_state=42,
    class_weight='balanced',
    n_jobs=-1
)
final_clf.fit(X_resampled, y_resampled)

# 3.5 最终模型评估
print("  评估最终模型...")
final_pred = final_clf.predict(X_test)
final_pred_proba = final_clf.predict_proba(X_test)[:, 1]

final_conf_matrix = confusion_matrix(y_test, final_pred)
tn, fp, fn, tp = final_conf_matrix.ravel()

final_accuracy = (tp + tn) / (tp + tn + fp + fn)
final_precision = tp / (tp + fp) if (tp + fp) > 0 else 0
final_recall = tp / (tp + fn) if (tp + fn) > 0 else 0
final_f1 = 2 * final_precision * final_recall / (final_precision + final_recall) if (final_precision + final_recall) > 0 else 0
final_specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

print(f"  最终混淆矩阵:\n{final_conf_matrix}")
print(f"  最终准确率: {final_accuracy:.4f}")
print(f"  最终精确率: {final_precision:.4f}")
print(f"  最终召回率: {final_recall:.4f}")
print(f"  最终F1分数: {final_f1:.4f}")
print(f"  最终特异性: {final_specificity:.4f}")

# 3.6 保存最终模型
joblib.dump(final_clf, f'{output_dir}/optimized_rf_classifier.pkl')
print(f"  最终模型已保存至: {output_dir}/optimized_rf_classifier.pkl")

# 3.7 比较基础模型和优化模型
comparison_metrics = pd.DataFrame({
    '基础模型': [accuracy, precision, recall, f1, specificity],
    '优化模型': [final_accuracy, final_precision, final_recall, final_f1, final_specificity]
}, index=['准确率', '精确率', '召回率', 'F1分数', '特异性'])

print("\n  模型比较:")
print(comparison_metrics)

comparison_metrics.to_csv(f'{output_dir}/model_comparison.csv')
print(f"  模型比较结果已保存至: {output_dir}/model_comparison.csv")

# 3.8 可视化模型比较
print("  可视化模型比较...")
plt.figure(figsize=(12, 8))
comparison_metrics.plot(kind='bar', rot=0)

if chinese_font is not None:
    plt.title('基础模型与优化模型性能比较', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('评估指标', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('分数', fontproperties=chinese_font, fontsize=12)
    plt.legend(prop=chinese_font)
else:
    plt.title('基础模型与优化模型性能比较', fontsize=16)
    plt.xlabel('评估指标', fontsize=12)
    plt.ylabel('分数', fontsize=12)

plt.ylim(0, 1)
plt.grid(True, axis='y')
plt.tight_layout()
plt.savefig(f'{output_dir}/model_comparison.png', dpi=300)
print(f"  保存图表：{output_dir}/model_comparison.png")
plt.close()


3. 优化模型参数
  定义参数搜索空间...
  执行随机参数搜索（这可能需要一些时间）...
Fitting 3 folds for each of 20 candidates, totalling 60 fits
  最佳F1分数: nan
  最佳参数:
    n_estimators: 100
    min_samples_split: 2
    min_samples_leaf: 2
    max_features: sqrt
    max_depth: 10
  使用最佳参数训练最终模型...
  评估最终模型...
  最终混淆矩阵:
[[203   1]
 [  0   0]]
  最终准确率: 0.9951
  最终精确率: 0.0000
  最终召回率: 0.0000
  最终F1分数: 0.0000
  最终特异性: 0.9951
  最终模型已保存至: dengue_classification_results/optimized_rf_classifier.pkl

  模型比较:
          基础模型      优化模型
准确率   0.995098  0.995098
精确率   0.000000  0.000000
召回率   0.000000  0.000000
F1分数  0.000000  0.000000
特异性   0.995098  0.995098
  模型比较结果已保存至: dengue_classification_results/model_comparison.csv
  可视化模型比较...
  保存图表：dengue_classification_results/model_comparison.png


<Figure size 1200x800 with 0 Axes>

In [9]:
# 4. 生成爆发预测结果和风险值
print("\n4. 生成爆发预测结果和风险值")

# 4.1 创建测试集预测结果表
test_results = test_df[['Region', 'Date', 'MonthNum', 'Year', 'Dengue_Cases', 'Outbreak']].copy()
test_results['Predicted_Outbreak'] = final_pred
test_results['Outbreak_Probability'] = final_pred_proba
test_results['Correct_Prediction'] = (test_results['Outbreak'] == test_results['Predicted_Outbreak']).astype(int)

# 保存测试集预测结果
test_results.to_csv(f'{output_dir}/test_predictions.csv', index=False)
print(f"  测试集预测结果已保存至: {output_dir}/test_predictions.csv")

# 4.2 可视化按月份和地区的预测性能
monthly_performance = test_results.groupby('MonthNum').agg({
    'Correct_Prediction': 'mean',
    'Outbreak': 'sum',
    'Predicted_Outbreak': 'sum'
}).reset_index()
monthly_performance['Month'] = monthly_performance['MonthNum'].apply(
    lambda x: ['一月', '二月', '三月', '四月', '五月', '六月', 
                '七月', '八月', '九月', '十月', '十一月', '十二月'][x-1])

plt.figure(figsize=(14, 7))
ax1 = plt.gca()
ax2 = ax1.twinx()

# 绘制正确预测率
ax1.plot(monthly_performance['Month'], monthly_performance['Correct_Prediction'], 
        'b-', marker='o', label='正确预测率')
ax1.set_ylim(0, 1)
ax1.set_ylabel('正确预测率', color='blue')

# 绘制爆发次数
bars = ax2.bar(monthly_performance['Month'], monthly_performance['Outbreak'], 
              alpha=0.3, color='red', label='实际爆发')
# 在柱上叠加预测爆发次数
ax2.bar(monthly_performance['Month'], monthly_performance['Predicted_Outbreak'], 
       alpha=0.3, color='orange', label='预测爆发')

if chinese_font is not None:
    plt.title('月度爆发预测性能', fontproperties=chinese_font, fontsize=16)
    ax1.set_xlabel('月份', fontproperties=chinese_font, fontsize=12)
    ax1.set_ylabel('正确预测率', fontproperties=chinese_font, fontsize=12)
    ax2.set_ylabel('爆发次数', fontproperties=chinese_font, fontsize=12)
    # 结合两个坐标轴的图例
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc='upper right', prop=chinese_font)
else:
    plt.title('月度爆发预测性能', fontsize=16)
    ax1.set_xlabel('月份', fontsize=12)
    ax1.set_ylabel('正确预测率', fontsize=12)
    ax2.set_ylabel('爆发次数', fontsize=12)
    # 结合两个坐标轴的图例
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc='upper right')

plt.xticks(rotation=45)
plt.grid(True, axis='y')
plt.tight_layout()
plt.savefig(f'{output_dir}/monthly_prediction_performance.png', dpi=300)
print(f"  保存图表：{output_dir}/monthly_prediction_performance.png")
plt.close()


4. 生成爆发预测结果和风险值
  测试集预测结果已保存至: dengue_classification_results/test_predictions.csv
  保存图表：dengue_classification_results/monthly_prediction_performance.png
